In [6]:
from load_results import load_result_dataset
import pandas as pd
import numpy as np
pn1 = 'full_fine_tuning_50epochs_edge_paper_final2'
pn2 = 'full_fine_tuning_50epochs_paper_final2'

final_data = load_result_dataset(pn1, pn2)
final_data = [{**d, 'ft_strategy': 'FFT (50 epochs)'} for d in final_data]

../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion_aesthetic-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion2B-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/deit_small_patch16_224.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/robust_resnet50_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/vit_small_patch16_224.augreg_in21k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/convnext_base.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/resnet50.a1_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/robust_vit_base_patch16_224_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_

In [2]:
df = pd.DataFrame(final_data)
for arch_size in ('small', 'medium', 'large'):
    df_curr = df[ df.model_size == arch_size ]
    nan_percentage = (df_curr.isna().sum().sum() / df_curr.size) * 100
    print("Percentage of NaN values, for {} architecture: {}".format(arch_size, nan_percentage) )
    

Percentage of NaN values, for small architecture: 3.125
Percentage of NaN values, for medium architecture: 0.0
Percentage of NaN values, for large architecture: 3.3854166666666665


In [7]:
import pandas as pd
import yaml

df = pd.DataFrame(final_data)
# df = df[df["model_size"].isin([  ])] # 'medium', 'large'
# df.to_csv("./{}.csv".format("result_with_missing_values_max"))

def get_restart_job(row):
    is_missing = lambda col: pd.isna(row[col])

    missing = {col: is_missing(col) for col in ['clean_acc', 'Linf_acc', 'L1_acc', 'L2_acc', 'common_acc']}

    if all(missing.values()):
        return "job1_hpo.sh"
    elif not missing['clean_acc'] and not missing['Linf_acc'] and missing['L1_acc']:
        return "job4_test_l1.sh"
    elif not missing['clean_acc'] and not missing['Linf_acc'] and not missing['L1_acc'] and missing['L2_acc']:
        return "job5_test_l2.sh"
    elif not missing['clean_acc'] and not missing['Linf_acc'] and not missing['L1_acc'] and not missing['L2_acc'] and missing['common_acc']:
        return "job6_test_common.sh"
    else:
        return None  # means no job needs to be restarted


df['restart_from'] = df.apply(get_restart_job, axis=1)

to_restart = df[df['restart_from'].notna()][['backbone', 'dataset', 'loss_function', 'restart_from', 'model_size']]
to_restart = to_restart[ to_restart.restart_from == "job1_hpo.sh" ]
to_restart.to_csv("./to_relaunch_rishika.csv")
to_restart

,backbone,dataset,loss_function,restart_from,model_size
71,coatnet_2_rw_224.sw_in12k_ft_in1k,stanford_cars,TRADES_v2,job1_hpo.sh,large
72,coatnet_2_rw_224.sw_in12k,stanford_cars,TRADES_v2,job1_hpo.sh,large
79,coat_tiny.in1k,stanford_cars,TRADES_v2,job1_hpo.sh,small
119,coat_tiny.in1k,caltech101,TRADES_v2,job1_hpo.sh,small


In [3]:
from process_database import process_grouped_df, process_rankings, compute_odds_ratio_by_group

size = "large"
grouped_df = process_grouped_df(final_data, size)
grouped_df = process_rankings(grouped_df)
grouped_df
# grouped_df.sort_values(by="sum", ascending=False)

Percentage of NaN values: 2.07%


dataset                                                                                                                                    TOTAL  \
metric                                                                                                                                     borda   
backbone                                        loss_function pre_training_strategy        model_type          model_size ft_strategy              
convnext_base.fb_in22k                          TRADES_v2     supervised                   fully convolutional large      FFT (50 epochs)  630.0   
CLIP-convnext_base_w-laion2B-s13B-b82K          TRADES_v2     self-supervised (multimodal) fully convolutional large      FFT (50 epochs)  590.0   
CLIP-convnext_base_w-laion_aesthetic-s13B-b82K  TRADES_v2     self-supervised (multimodal) fully convolutional large      FFT (50 epochs)  669.0   
swin_base_patch4_window7_224.ms_in22k_ft_in1k   TRADES_v2     supervised                   fully attention     large      FFT (50 epochs)  639.0   
eva02_base_patch14_224.mim_in22k                TRADES_v2     self-supervised              fully attention     large      FFT (50 epochs)  640.0   
robust_convnext_base                            TRADES_v2     supervised (robust)          fully convolutional large      FFT (50 epochs)  627.0   
convnext_base.fb_in22k_ft_in1k                  TRADES_v2     supervised                   fully convolutional large      FFT (50 epochs)  498.0   
vit_base_patch16_clip_224.laion2b_ft_in1k       TRADES_v2     hybrid                       fully attention     large      FFT (50 epochs)  529.0   
convnext_base.fb_in1k                           TRADES_v2     supervised                   fully convolutional large      FFT (50 epochs)  506.0   
convnext_base.clip_laion2b_augreg_ft_in12k_in1k TRADES_v2     hybrid                       fully convolutional large      FFT (50 epochs)  523.0   
vit_base_patch16_clip_224.laion2b               TRADES_v2     self-supervised (multimodal) fully attention     large      FFT (50 epochs)  492.0   
swin_base_patch4_window7_224.ms_in22k_ft_in1k   CLASSIC_AT    supervised                   fully attention     large      FFT (50 epochs)  552.0   
vit_base_patch16_224.augreg_in1k                TRADES_v2     supervised                   fully attention     large      FFT (50 epochs)  394.0   
                                                CLASSIC_AT    supervised                   fully attention     large      FFT (50 epochs)  325.0   
CLIP-convnext_base_w-laion2B-s13B-b82K          CLASSIC_AT    self-supervised (multimodal) fully convolutional large      FFT (50 epochs)  508.0   
vit_base_patch16_224.mae                        TRADES_v2     self-supervised              fully attention     large      FFT (50 epochs)  363.0   
convnext_base.fb_in22k_ft_in1k                  CLASSIC_AT    supervised                   fully convolutional large      FFT (50 epochs)  543.0   
vit_base_patch16_224.mae                        CLASSIC_AT    self-supervised              fully attention     large      FFT (50 epochs)  305.0   
CLIP-convnext_base_w-laion_aesthetic-s13B-b82K  CLASSIC_AT    self-supervised (multimodal) fully convolutional large      FFT (50 epochs)  446.0   
robust_convnext_base                            CLASSIC_AT    supervised (robust)          fully convolutional large      FFT (50 epochs)  489.0   
robust_vit_base_patch16_224                     TRADES_v2     supervised (robust)          fully attention     large      FFT (50 epochs)  340.0   
vit_base_patch16_224.augreg_in21k               TRADES_v2     supervised                   fully attention     large      FFT (50 epochs)  259.0   
eva02_base_patch14_224.mim_in22k                CLASSIC_AT    self-supervised              fully attention     large      FFT (50 epochs)  398.0   
coatnet_2_rw_224.sw_in12k_ft_in1k               TRADES_v2     supervised                   hybrid              large      FFT (50 epochs)  294.0   
vit_

In [4]:
sub = grouped_df.reset_index()
sub[ "caltech101"]

metric,L1_acc,L2_acc,Linf_acc,borda,clean_acc,common_acc,geom,sum,volume_pre_training_data
0,0.860000,0.888000,0.841000,NaN,0.959000,NaN,0.615922,3.548000,14197122
1,0.803000,0.854667,0.806000,NaN,0.937000,NaN,0.518307,3.400667,2320000000
2,0.794000,0.856000,0.809333,NaN,0.952000,NaN,0.523671,3.411333,900000000
3,0.836000,0.866667,0.797000,NaN,0.938000,NaN,0.541651,3.437667,15478289
4,0.844000,0.890000,0.824000,NaN,0.949000,NaN,0.587389,3.507000,14197122
5,0.748000,0.892000,0.878000,NaN,0.964000,NaN,0.564726,3.482000,1281167
6,0.857000,0.898000,0.842000,NaN,0.958000,NaN,0.620776,3.555000,15478289
7,0.776000,0.820000,0.774000,68.0,0.919000,0.832000,0.376578,4.121000,2321281167
8,0.838000,0.871000,0.812000,NaN,0.958000,NaN,0.567785,3.479000,1281167
9,0.812000,0.843000,0.802667,NaN,0.920000,NaN,0.505483,3.377667,2330281167


In [4]:
from process_database import process_grouped_df, process_rankings, compute_odds_ratio_by_group

grouped_df = process_grouped_df(final_data, size="small")
grouped_df = process_rankings(grouped_df)
grouped_df

Percentage of NaN values: 3.75%


dataset                                                                                TOTAL  \
metric                                                                                 borda   
backbone                      loss_function pre_training_strategy model_type                   
edgenext_small.usi_in1k       TRADES_v2     supervised            hybrid               365.0   
deit_tiny_patch16_224.fb_in1k TRADES_v2     supervised            fully attention      256.0   
regnetx_004.pycls_in1k        CLASSIC_AT    supervised            fully convolutional  252.0   
                              TRADES_v2     supervised            fully convolutional  158.0   
mobilenetv3_large_100.ra_in1k TRADES_v2     supervised            fully convolutional  177.0   
edgenext_small.usi_in1k       CLASSIC_AT    supervised            hybrid               275.0   
mobilenetv3_large_100.ra_in1k CLASSIC_AT    supervised            fully convolutional  206.0   
deit_tiny_patch16_224.fb_in1k CLASSIC_AT    supervised            fully attention      167.0   
mobilevit-small               CLASSIC_AT    supervised            hybrid               177.0   
efficientnet-b0               CLASSIC_AT    supervised            fully convolutional  124.0   
coat_tiny.in1k                TRADES_v2     supervised            hybrid                58.0   
efficientnet-b0               TRADES_v2     supervised            fully convolutional   91.0   
mobilevit-small               TRADES_v2     supervised            hybrid               139.0   
coat_tiny.in1k                CLASSIC_AT    supervised            hybrid               105.0   

dataset                                                                                             \
metric                                                                                nan_geom_cnt   
backbone                      loss_function pre_training_strategy model_type                         
edgenext_small.usi_in1k       TRADES_v2     supervised            hybrid                         0   
deit_tiny_patch16_224.fb_in1k TRADES_v2     supervised            fully attention                0   
regnetx_004.pycls_in1k        CLASSIC_AT    supervised            fully convolutional            0   
                              TRADES_v2     supervised            fully convolutional            0   
mobilenetv3_large_100.ra_in1k TRADES_v2     supervised            fully convolutional            0   
edgenext_small.usi_in1k       CLASSIC_AT    supervised            hybrid                         1   
mobilenetv3_large_100.ra_in1k CLASSIC_AT    supervised            fully convolutional            0   
deit_tiny_patch16_224.fb_in1k CLASSIC_AT    supervised            fully attention                0   
mobilevit-small               CLASSIC_AT    supervised            hybrid                         0   
efficientnet-b0               CLASSIC_AT    supervised            fully convolutional            0   
coat_tiny.in1k                TRADES_v2     supervised            hybrid                         5   
efficientnet-b0               TRADES_v2     supervised            fully convolutional            0   
mobilevit-small               TRADES_v2     supervised            hybrid                         1   
coat_tiny.in1k                CLASSIC_AT    supervised            hybrid                         2   

dataset                                                                                            \
metric                                                                                nan_sum_cnt   
backbone                      loss_function pre_training_strategy model_type                        
edgenext_small.usi_in1k       TRADES_v2     supervised            hybrid                        0   
deit_tiny_patch16_224.fb_in1k TRADES_v2     supervised            fully attention               0   
regnetx_004.pycls_in1k        CLASSIC_AT    supervised            fully convolutional           0   
                          

In [5]:

from process_database import process_grouped_df, process_rankings, compute_odds_ratio_by_group
import pandas as pd

odds_loss_list = []
odds_architecture_list = []
odds_pretraining_list = []

for size_id,size in [ (0,'small'),(1,'medium'),(2,'large')]:
    grouped_df = process_grouped_df(final_data, size)
    grouped_df = process_rankings(grouped_df)

    odds_df = grouped_df['TOTAL']
    odds_df = odds_df.reset_index()
    odds_df.to_csv("./odds_ratio.csv")

    # Define tier1 threshold based on the number of rows
    n_rows = odds_df.shape[0]
    top_k = int(np.ceil(0.33 * n_rows))  # top 10% as tier1

    # Apply tier1 flag
    odds_df["in_tier1"] = 0
    odds_df.loc[odds_df.sort_values(by="score_sum", ascending=False).head(top_k).index, "in_tier1"] = 1

    # Compute odds ratios for model_type and loss_function
    odds_model_type = compute_odds_ratio_by_group(odds_df, "model_type")
    odds_pretrain_type = compute_odds_ratio_by_group(odds_df, "pre_training_strategy")
    odds_loss_function = compute_odds_ratio_by_group(odds_df, "loss_function")
    odds_loss_function['size'] = size
    odds_pretrain_type['size'] = size
    odds_model_type['size'] = size

    odds_loss_list.append(odds_loss_function)
    odds_architecture_list.append(odds_model_type)
    odds_pretraining_list.append(odds_pretrain_type)

combined_odds_loss = pd.concat(odds_loss_list, axis=0)
combined_odds_architecture = pd.concat(odds_architecture_list, axis=0)
combined_odds_pretraining = pd.concat(odds_pretraining_list, axis=0)

combined_odds_pretraining = combined_odds_pretraining.fillna(0)
combined_odds_pretraining = combined_odds_pretraining[combined_odds_pretraining["size"].isin(["medium", "large"])]


Percentage of NaN values: 3.75%
Percentage of NaN values: 3.75%
Percentage of NaN values: 3.75%


In [6]:
import plotly.express as px

# Create mapping from numeric size to categorical label
size_map = {"small":0, "medium":1, "large":2}

# Convert numeric size columns to categorical labels
combined_odds_loss["size_label"] = combined_odds_loss["size"].map(size_map)
combined_odds_architecture["size_label"] = combined_odds_architecture["size"].map(size_map)
combined_odds_pretraining["size_label"] = combined_odds_pretraining["size"].map(size_map)

plots = [
    ("loss_function", combined_odds_loss),
    ("model_type", combined_odds_architecture),
    ("pre_training_strategy", combined_odds_pretraining)
]


figs = []

for var, plot_df in plots:

    plot_df = plot_df.copy()
    plot_df["odds_ratio"] = plot_df["odds_ratio"].replace(0, 0)

    # plot_df["text_label"] = plot_df.apply(
    #     lambda row: (
    #         f'{int(row["odds_ratio"])}' if row["odds_ratio"] == int(row["odds_ratio"])
    #         else f'{row["odds_ratio"]:.2f}'
    #     ) + f'  (p:{row["p_value"]:.3f})',
    #     axis=1
    # )
    plot_df["text_label"] = plot_df.apply(
        lambda row: f'{int(row["odds_ratio"])}' if row["odds_ratio"] == int(row["odds_ratio"])
        else f'{row["odds_ratio"]:.2f}', axis=1 )
    plot_df["text_label"] = plot_df.apply(
        lambda row: f'{row["text_label"]}<br>(p={row["p_value"]:.3f})', axis=1)
            
    fig = px.bar(
        plot_df,
        x="size",
        y="odds_ratio",
        color=var,
        barmode="group",
        text="text_label",
        labels={"size_label": "Model Size", "odds_ratio": "Odds Ratio"},
    )

    fig.update_layout(yaxis=dict(range=[0, 25]),)

    fig.update_traces(
        textposition="outside",
        textangle=-90,  # Rotate text vertically
        textfont=dict(
            color="black",
            family="Arial"
        ),
        cliponaxis=False
    )

    if var == 'loss_function':
        fig.update_layout(
        uniformtext_minsize=11,  # or your desired minimum font size
        uniformtext_mode='show',  # force showing all text
        )
    else:
        fig.update_layout(
        uniformtext_minsize=7,  # or your desired minimum font size
        uniformtext_mode='show',  # force showing all text
        )



    fig.update_layout(
            margin=dict(l=0, r=0, t=0, b=0),  # remove all outer margins
            width=450,    # in pixels (~2.5 inches at 96 DPI)
            height=400,    # in pixels (~1.875 inches)
            yaxis_title="Odds Ratio",
            xaxis_title="Model Size",
            legend_title_text=var,
            
            # White background
            plot_bgcolor="white",
            paper_bgcolor="white",
            
            # Light grey grid
            xaxis=dict(showgrid=False, gridcolor="lightgrey"),
            yaxis=dict(showgrid=True, gridcolor="lightgrey"),
            
            # Legend inside figure
            legend=dict(    
                font=dict(size=12),  # Adjust font size here 
                yanchor="top",
                xanchor="right",
                bgcolor="rgba(255,255,255,1)",  # semi-transparent white box
                bordercolor="lightgrey",
                borderwidth=1
            )
        )
    fig.show()
    fig.write_image("./paper_figures/oddsratio_{}_{}.png".format(pn2,var), scale=3  )
    
    

In [14]:

from matplotlib import cm
from matplotlib.colors import to_hex
import re


import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px


import pandas as pd

df1 = grouped_df.copy()

metrics_to_sum = ["L1_acc", "L2_acc", "Linf_acc", "clean_acc", "common_acc"]

for metric in metrics_to_sum:
    metric_cols = [col for col in df1.columns if col[1] == metric]
    df1[f'sum_{metric}'] = df1[metric_cols].sum(axis=1)

df_per_row_sums = df1[[f'sum_{m}' for m in metrics_to_sum]]

df_per_row_sums.columns = df_per_row_sums.columns.get_level_values(0)

df_per_row_sums.columns.name = 'metric'

df_per_row_sums


# Your model type mapping
model_type_map = {
    'convnext_base': "fully convolutional",
    'convnext_tiny': "fully convolutional",
    'deit_small': "fully attention",
    'vit_base': "fully attention",
    'vit_small': "fully attention",
    'resnet50': "fully convolutional",
    'eva02_base': "fully attention",
    'eva02_tiny': "fully attention",
    'swin_base': "fully attention",
    'swin_tiny': "fully attention",
    'coatnet_0': "hybrid",
    'coatnet_2': "hybrid",
    'regnetx_004': "fully convolutional",
    'efficientnet-b0': "fully convolutional", 
    'deit_tiny': "fully attention",
    'mobilevit-small': "hybrid",
    'mobilenetv3': "fully convolutional",
    'edgenext_small': "fully convolutional",
    'coat_tiny': "hybrid",
}

# Colormap for each model type
type_to_cmap = {
    'fully convolutional': cm.Purples,     # deep violet → lavender
    'fully attention': cm.Oranges,         # strong orange → light peach
    'hybrid': cm.Greens                    # forest → mint
}

# Extract base backbone name from full backbone string
def extract_base_name(backbone_name):
    for base in model_type_map:
        if base in backbone_name:
            return base
    return 'unknown'

# Get all unique backbones
backbones = sorted(set(backbone for backbone, _, _, _ in df_per_row_sums.index))

# Map backbones to model type
backbone_model_type = {b: model_type_map.get(extract_base_name(b), 'unknown') for b in backbones}

# Assign gradient color per backbone based on model type
color_map = {}
for model_type in ['fully convolutional', 'fully attention', 'hybrid']:
    bks = [bk for bk in backbones if backbone_model_type[bk] == model_type]
    n = len(bks)
    for i, bk in enumerate(bks):
        start, end = 0.5, 0.9  # avoid very light or very dark edges
        position = start + (end - start) * (i / max(n - 1, 1))
        color = to_hex(type_to_cmap[model_type](position))
        color_map[bk] = color


# Define metrics
metrics = ['sum_L1_acc', 'sum_L2_acc', 'sum_Linf_acc', 'sum_clean_acc', 'sum_common_acc']

# Create a list of unique backbones (regardless of loss)
unique_backbones = sorted(set(backbone for backbone, _, _, _ in df_per_row_sums.index))

# # Assign a unique color to each backbone
# color_palette = px.colors.qualitative.Plotly
# color_map = dict(zip(unique_backbones, color_palette * (len(unique_backbones) // len(color_palette) + 1)))

# Define line styles per loss function
line_styles = {
    'CLASSIC_AT': 'solid',
    'TRADES_v2': 'dot'
}

# Create the radar plot

from plotly.colors import qualitative
color_palette = qualitative.Bold

# Get unique backbones
unique_backbones = sorted(set(backbone for backbone, _, _, _ in df_per_row_sums.index))

# Assign colors per backbone
# color_map = dict(zip(unique_backbones, color_palette * (len(unique_backbones) // len(color_palette) + 1)))

fig = go.Figure()

for (backbone, loss, _, _), row in df_per_row_sums.iterrows():
    fig.add_trace(go.Scatterpolar(
        r=row.tolist() + [row.tolist()[0]],  # Close the loop
        theta=metrics + [metrics[0]],
        mode='lines',
        name=f"{backbone} ({loss})",
        line=dict(
            width=2,
            color=color_map[backbone],
            dash=line_styles.get(loss, 'solid')
        )
    ))

fig.update_layout(
    width=500, 
    height=500,
    paper_bgcolor='white',
    plot_bgcolor='white',
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 6],
            showline=True,
            linewidth=1,
            linecolor="lightgrey",
            gridcolor="lightgrey",
            gridwidth=0.5,
        ),
        angularaxis=dict(
            direction="clockwise",
            showline=True,
            linewidth=1,
            linecolor="lightgrey",
            gridcolor="lightgrey",
            gridwidth=0.5,
        ),
        bgcolor='white'
    ),
    legend=dict(
        x=0,
        y=1.0,
        xanchor='left',
        yanchor='top',
        bgcolor='rgba(255,255,255,0.9)',
        bordercolor='lightgrey',
        borderwidth=1,
        font=dict(size=11),
    ),
    showlegend=False,
    font=dict(color='black'),
    margin=dict(l=25, r=25, t=25, b=25)  # <<< reduced margins
)

fig.show()

fig.write_image("./radar_plot_{}_{}.png".format(size, pn2), scale=3  )                # upscale for higher DPI (1 = default)